# SparkSession

U uvodnom tekstu smo ukratko spomenuli Spark ulaznu točku zvanu SparkContext koja predstavlja konekciju sa Spark klasterom - SparkSession je nadskup toga. Okružuje SparkContext i pruža mogućnost interakcije sa Spark SQL API-jem koji sadrži DF koji ćemo koristiti u većini naših programa. 

Prije pokretanja PySpark koda potrebno je kreirati SparkSession instancu. Dok je pokrenuta možete dohvatiti pregled vašeg Spark klastera i svih operacija na http://localhost:4040.

SparkSession objektu ćete dodijeliti određene parametre kako biste postavili i konfigurirali Spark aplikaciju - neki od parametara uključuju naziv aplikacije, URL i postavke alokacije memorije.Tako naša aplikacija postavlja sljedeće parametre:
* `builder` - objekt za kreiranje i konfiguraciju SparkSessiona
* `master("local[*]")` - lokalno izvršavanje aplikacije gdje * označava izrvšavanje aplikacije na svim dostupnim jezgrama - ovdje možete upisati i broj jezgri
* `appName("SparkVjezba")` - ime aplikacije
* `getOrCreate()` - kreira novi SparkSession ako ne postoji ili vraća postojeći ako se već izvodi te time osigurava da je samo jedan SparkSession objekt kreiran za aplikaciju što je važno za efikasnu upotrebu resursa. 

In [1]:
import findspark
from pyspark.sql import SparkSession

findspark.init()

# added config for postgresql driver for lab4
spark = SparkSession.builder.master("local[*]") \
    .appName("SparkVjezba") \
    .config("spark.jars", "/mnt/c/apps/opt/spark-3.3.2-bin-hadoop3/jars/postgresql-42.6.0.jar") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/05/22 11:10:37 WARN Utils: Your hostname, mkoscec resolves to a loopback address: 127.0.1.1; using 172.25.217.28 instead (on interface eth0)
23/05/22 11:10:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/05/22 11:10:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# DataFrame

Ako ste korisitli Pandas biblioteku u Pythonu vjerojatno ste upoznati s konceptom DataFramea. Ipak, važno je naglasiti da DataFrame u Pandasu i Sparku nisu isti; neke od razlika su:
| Aspekt | Pandas DataFrame | Spark DataFrame |
|:--------:|:-----------------:|:----------------:|
| Izvršavanje | Manji setovi podataka i izvršavanje na jednom stroju | Veliki setovi podataka i izvršavanje na više storjeva |
| Evaluacija | Agilna | Lijena |
| Brzina | Brži za manje skupove podataka| Brži za veće skupove podataka |
| Izvori podataka | Zahtijeva dodatne bibioloteke za čitanje i pisanje raznih izvora podataka | Ugrađena potpora za čitanje i pisanje raznih izvora podataka| |

<br/>
Dataframe u Sparku možete shvatiti kao tablicu s redovima i stupcima koja sadrži veliku količinu podataka. Obrada podataka u Dataframeu se vrši korištenjem transformacija i akcija, a rezultat se može spremiti u novi Dataframe. Dataframe je također vrlo fleksibilan i može se koristiti za razne obrade podataka kao što su spajanje, filtriranje, sortiranje, agregacija i grupiranje podataka. 

## Kreiranje i korištenje DataFramea 

In [2]:
# kreacija DataFramea
data = [(1,2), (3,4)]
schema = "neparni int, parni int"
df_list = spark.createDataFrame(data, schema)

# prikaz DataFramea
df_list.show()

# prikaz scheme DataFramea
df_list.printSchema()

+-------+-----+
|neparni|parni|
+-------+-----+
|      1|    2|
|      3|    4|
+-------+-----+

root
 |-- neparni: integer (nullable = true)
 |-- parni: integer (nullable = true)



In [3]:
# selekcija stupaca
df_list.select(["neparni"]).show()

+-------+
|neparni|
+-------+
|      1|
|      3|
+-------+



In [4]:
# dodavanje stupaca
from pyspark.sql.functions import col

df_list.withColumn("zbroj", col("neparni") + col("parni")).show()

+-------+-----+-----+
|neparni|parni|zbroj|
+-------+-----+-----+
|      1|    2|    3|
|      3|    4|    7|
+-------+-----+-----+



Prilikom stvaranja DF-a ukoliko nema zaglavlja (ili se ona ne čitaju) zadana vrijednost ukoliko se DF stvara iz CSV datoteke je _cn - gdje n predstavlja broj stupca počevši od nule, dok ukoliko se stvara iz liste zadana vrijednost je _n - gdje je n ekvivalent prethodnom n-u.

In [5]:
# preimenovanje stupaca
df_list_n = spark.createDataFrame(data)
df_list_n.withColumnRenamed("_1", "neparni") \
         .withColumnRenamed("_2", "parni") \
         .show()

+-------+-----+
|neparni|parni|
+-------+-----+
|      1|    2|
|      3|    4|
+-------+-----+



## Spark SQL i Window funkcije

Windowing u Sparku omogućuje obradu podataka u prozorima; odnosno, funkcije se mogu primijeniti i na više elemenata, a ne samo na pojedinačni element.

In [6]:
employee_data = (("John", "Engineering", 7000),
                 ("Jane", "Marketing", 5000),
                 ("Bob", "Engineering", 6000),
                 ("Mary", "Sales", 5500),
                 ("Alex", "Marketing", 4500),
                 ("Mike", "Engineering", 7500),
                 ("Julie", "Sales", 6000),
                 ("Sam", "Finance", 6500),
                 ("Lisa", "Marketing", 4500),
                 ("Tom", "Engineering", 8000))

columns = ["name", "department", "salary"]

df_employees = spark.createDataFrame(employee_data, columns)

### Funkcije rangiranja

Funkcija row_number koristi se za dodjelu slijednog broja retka koji počinje od 1 - svaki broj koristi samo jednom neovisno o tome je li došlo do izjenačenja.

In [7]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window_partition = Window.partitionBy("department").orderBy("salary")

df_employees.withColumn("row_number", row_number().over(window_partition)).show()

+-----+-----------+------+----------+
| name| department|salary|row_number|
+-----+-----------+------+----------+
|  Bob|Engineering|  6000|         1|
| John|Engineering|  7000|         2|
| Mike|Engineering|  7500|         3|
|  Tom|Engineering|  8000|         4|
|  Sam|    Finance|  6500|         1|
| Alex|  Marketing|  4500|         1|
| Lisa|  Marketing|  4500|         2|
| Jane|  Marketing|  5000|         3|
| Mary|      Sales|  5500|         1|
|Julie|      Sales|  6000|         2|
+-----+-----------+------+----------+



Funkcije rank i dense_rank dodjeljuju rang rezultatu unutar prozora. Dok rank ostavlja prazna mjesta ako dolazi do izjednačenja, dense_rank preskače praznine.

In [8]:
from pyspark.sql.functions import rank, dense_rank

df_employees.withColumn("rank", rank().over(window_partition)) \
            .withColumn("dense_rank", dense_rank().over(window_partition)) \
            .show()

+-----+-----------+------+----+----------+
| name| department|salary|rank|dense_rank|
+-----+-----------+------+----+----------+
|  Bob|Engineering|  6000|   1|         1|
| John|Engineering|  7000|   2|         2|
| Mike|Engineering|  7500|   3|         3|
|  Tom|Engineering|  8000|   4|         4|
|  Sam|    Finance|  6500|   1|         1|
| Alex|  Marketing|  4500|   1|         1|
| Lisa|  Marketing|  4500|   1|         1|
| Jane|  Marketing|  5000|   3|         2|
| Mary|      Sales|  5500|   1|         1|
|Julie|      Sales|  6000|   2|         2|
+-----+-----------+------+----+----------+



Funkcija percent_rank namijenjena je za izračunavanje percentila za svaki redak unutar prozora.

In [9]:
from pyspark.sql.functions import percent_rank

df_employees.withColumn("percent_rank", percent_rank().over(window_partition)).show()

+-----+-----------+------+------------------+
| name| department|salary|      percent_rank|
+-----+-----------+------+------------------+
|  Bob|Engineering|  6000|               0.0|
| John|Engineering|  7000|0.3333333333333333|
| Mike|Engineering|  7500|0.6666666666666666|
|  Tom|Engineering|  8000|               1.0|
|  Sam|    Finance|  6500|               0.0|
| Alex|  Marketing|  4500|               0.0|
| Lisa|  Marketing|  4500|               0.0|
| Jane|  Marketing|  5000|               1.0|
| Mary|      Sales|  5500|               0.0|
|Julie|      Sales|  6000|               1.0|
+-----+-----------+------+------------------+



Funkcija prozora ntile koristi se za podjelu rezultata unutar prozora u n jednakih dijelova.

In [10]:
from pyspark.sql.functions import ntile

df_employees.withColumn("ntile", ntile(2).over(window_partition)).show()

+-----+-----------+------+-----+
| name| department|salary|ntile|
+-----+-----------+------+-----+
|  Bob|Engineering|  6000|    1|
| John|Engineering|  7000|    1|
| Mike|Engineering|  7500|    2|
|  Tom|Engineering|  8000|    2|
|  Sam|    Finance|  6500|    1|
| Alex|  Marketing|  4500|    1|
| Lisa|  Marketing|  4500|    1|
| Jane|  Marketing|  5000|    2|
| Mary|      Sales|  5500|    1|
|Julie|      Sales|  6000|    2|
+-----+-----------+------+-----+



### Analitičke funkcije 

Funkcija cume_dist koristi se za izračun kumulativne distribucije vrijednosti unutar svakog prozora.

In [11]:
from pyspark.sql.functions import cume_dist

df_employees.withColumn("cume_dist", cume_dist().over(window_partition)).show()

+-----+-----------+------+------------------+
| name| department|salary|         cume_dist|
+-----+-----------+------+------------------+
|  Bob|Engineering|  6000|              0.25|
| John|Engineering|  7000|               0.5|
| Mike|Engineering|  7500|              0.75|
|  Tom|Engineering|  8000|               1.0|
|  Sam|    Finance|  6500|               1.0|
| Alex|  Marketing|  4500|0.6666666666666666|
| Lisa|  Marketing|  4500|0.6666666666666666|
| Jane|  Marketing|  5000|               1.0|
| Mary|      Sales|  5500|               0.5|
|Julie|      Sales|  6000|               1.0|
+-----+-----------+------+------------------+



Funkcija lag se koristi za dobivanje n-tog prethodnika reda unutar prozora, dok ćete funkciju lead iskoristiti kada želite dobit n-tog sljedbenika reda.

In [12]:
from pyspark.sql.functions import lag, lead

df_employees.withColumn("lag", lag("salary", 1).over(window_partition)) \
            .withColumn("lead", lead("salary", 1).over(window_partition)) \
            .show()

+-----+-----------+------+----+----+
| name| department|salary| lag|lead|
+-----+-----------+------+----+----+
|  Bob|Engineering|  6000|null|7000|
| John|Engineering|  7000|6000|7500|
| Mike|Engineering|  7500|7000|8000|
|  Tom|Engineering|  8000|7500|null|
|  Sam|    Finance|  6500|null|null|
| Alex|  Marketing|  4500|null|4500|
| Lisa|  Marketing|  4500|4500|5000|
| Jane|  Marketing|  5000|4500|null|
| Mary|      Sales|  5500|null|6000|
|Julie|      Sales|  6000|5500|null|
+-----+-----------+------+----+----+



### Agregatne funkcije



In [13]:
from pyspark.sql.functions import avg, sum, min, max

window_aggregation  = Window.partitionBy("department")

df_employees.withColumn("row", row_number().over(window_partition)) \
            .withColumn("avg", avg(col("salary")).over(window_aggregation)) \
            .withColumn("sum", sum(col("salary")).over(window_aggregation)) \
            .withColumn("min", min(col("salary")).over(window_aggregation)) \
            .withColumn("max", max(col("salary")).over(window_aggregation)) \
            .where(col("row") == 1).select("department", "avg", "sum", "min", "max") \
            .show()

+-----------+-----------------+-----+----+----+
| department|              avg|  sum| min| max|
+-----------+-----------------+-----+----+----+
|Engineering|           7125.0|28500|6000|8000|
|    Finance|           6500.0| 6500|6500|6500|
|  Marketing|4666.666666666667|14000|4500|5000|
|      Sales|           5750.0|11500|5500|6000|
+-----------+-----------------+-----+----+----+



## Čitanje iz datoteke
PySpark posjeduje kapacitet otkrivanja scheme - možete uključiti tu opciju postavljajući inferSchema na True. Ovaj opcionalni parametar tjera PySpark da prođe kroz podatke dvaput - prvi put kako bi postavio tip svakog stupca i drugi put da bi učitao podatke. Time učitavanje traje duže, ali nam pomaže izbjeći ručno pisanje scheme. Preuzmite skup podataka sa sljedeće poveznice https://www.kaggle.com/datasets/crawford/80-cereals te ga stavite u mapu u kojoj se nalazi vaša bilježnica.

In [14]:
# Učitavanje datoteke sa zaglavljem i otkrivanjem scheme
df_cereal = spark.read.format("csv").option("header", True).option("inferSchema", True).load("cereal.csv")

# Prikaz samo prva tri reda
df_cereal.show(3)

# Primijetite tipove 
df_cereal.printSchema()

+-----------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|             name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+-----------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|        100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|         All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
+-----------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
only showing top 3 rows

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- calories: integer 

## Agregacije nad grupiranim stupcem
Ako želite grupirati stupce možete se poslužiti transformacijom `groupby("imestupca")`. Metoda `agg()` (agg iz "aggregation") prihvaća jednu ili više agregatnih funkcija iz modula `pyspark.sql.functions`. U primjeru ispod računamo prosječni broj kalorija po tipu žitarica te novi stupac imenujemo metodom `alias("imestupca")` koji bi u protivnom nosio naziv prema funkcijama izvedenim u `agg()` metodi - u konkretnom slučaju "round(avg(calories), 2)".

In [15]:
from pyspark.sql.functions import round

df_cereal.groupBy("type") \
    .agg(round(avg("calories"), 2).alias("avg_calories")) \
    .show()


+----+------------+
|type|avg_calories|
+----+------------+
|   C|      107.16|
|   H|       100.0|
+----+------------+



## Važno! 
Prije nastavka rješavanja obavite zadatak iz poglavlja "*Povezivanje PySparka s Kafkom*". Obratite pozornost na stupce u datoteci - ako ste čitali cijelu vrijednost cijeli redak će vam biti zapisan pod jedan stupac. Razdvojite datoteku po stupcima prije nastavka - npr. spremanjem datoteke nanovo u CSV formatu.
## Dnevni prosjek 
Vaš zadatak je prikazati kretanje prosječne dnevne cijene kroz proteklih sto dana koristeći DataFrame kao strukturu podataka. Ispišite vaše rješenje na ekran - poredajte elemente silazno po prosječnoj cijeni zaokruženoj na jednu decimalu te im dodijelite rang. Možete koristiti `from_unixtime` iz `pyspark.sql.functions` modula za izvršenje zadatka.

In [16]:
from pyspark.sql.functions import from_unixtime, date_format, avg, desc
from pyspark.sql.window import Window

df_bitcoin = spark.read.format("csv").option("header", False).option("inferSchema", True).load("bitcoin.csv")

df_bitcoin = df_bitcoin.withColumnRenamed("_c0", "timestamp") \
    .withColumnRenamed("_c1", "value")

df_bitcoin = df_bitcoin.withColumn("date", date_format(from_unixtime(df_bitcoin["timestamp"]/1000), "yyyy-MM-dd"))

window = Window.partitionBy("date")

df_bitcoin.groupBy("date") \
    .agg(round(avg("value"), 2).alias("avg_price")) \
    .withColumn("row", row_number().over(Window.orderBy(desc("avg_price")))) \
    .limit(100) \
    .show(100)


23/05/22 11:10:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/22 11:10:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/22 11:10:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/22 11:10:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/22 11:10:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/22 11:10:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+---------

# Resilient Distributed Dataset (RDD) 
Postoje dva načina kako kreirati RDD: 
* paralelizacijom postojeće kolekcije - pozivanjem metode parallelize
* referenciranjem skupa podataka u vanjsokom sustavu za pohranu - pozivanjem metode textfile koja kao argument prihvaća put do datoteke


RDD se poziva nad SparkContextom koji možete izvući iz SparkSession objekta.

In [17]:
# SparkContext objekt iz SparkSessiona
sc = spark.sparkContext

# Paralelizacija postojeće kolekcije
rdd_array = sc.parallelize([1, 2, 3])

# Učitavanje iz datoteke - zamijenite naziv CSV datoteke sa svojim
rdd_csv = sc.textFile("bitcoin.csv")

## Osnovne akcije nad RDD-ovima 
Metoda `collect()` dohvaća cijeli RDD (primijetite kako je svaki redak pročitan kao jedan element iako sadrži dva podatka). 

In [18]:
rdd_csv.collect()

['1677754800000,23392.4024658030736446',
 '1677758400000,23412.8301319667936481',
 '1677762000000,23369.1657988242457019',
 '1677765600000,23317.9928455364540223',
 '1677769200000,23304.3620881745440072',
 '1677772800000,23371.0505556610832414',
 '1677776400000,23294.6043278283688753',
 '1677780000000,23289.4984095429347556',
 '1677783600000,23443.2121449701068961',
 '1677787200000,23478.0257965128492930',
 '1677790800000,23463.8413121360464792',
 '1677794400000,23431.7409495071492525',
 '1677798000000,23476.4063053381767990',
 '1677801600000,23472.5406312534858921',
 '1677805200000,23422.7917406744163382',
 '1677808800000,22247.0716362180135542',
 '1677812400000,22381.7864746646745348',
 '1677816000000,22352.7524478982899446',
 '1677819600000,22402.0371422146906710',
 '1677823200000,22366.0191666461221423',
 '1677826800000,22393.6259234369690590',
 '1677830400000,22396.1993914854484418',
 '1677834000000,22447.0850612010894038',
 '1677837600000,22386.1993386281541886',
 '1677841200000,

Metoda `count()` pokazuje broj elemenata u RDD-u.

In [19]:
rdd_csv.count()

16380

Kako biste uzeli određeni broj elemenata (po redu) potrebno je pozvati `take(n)` gdje n ukazuje koji je to broj elemenata:

In [20]:
rdd_csv.take(2)

['1677754800000,23392.4024658030736446',
 '1677758400000,23412.8301319667936481']

## Transformacije nad RDD-ovima

Tranformacija filter se koristi kako bi se zadržali elementi koji ispunjavaju određeni uvjet; nad svakim elementom se poziva funkcija te ako je odgovor na tu funkciju True taj element se zadržava.

In [21]:
veci_od_jedan = rdd_array.filter(lambda x: x > 1)
veci_od_jedan.collect()

[2, 3]

Koristeći map transformaciju možete nad svakim elementom RDD-a izvesti određenu funkciju. Primjerice, ako imate csv datoteku u kojoj imate više stupaca - te elemente možete pročitati pozivajući funkciju koja će razdvojiti vaš redak u elemente po delimiteru u vašoj datoteci.

Kao vježbu napišite naredbu s lambda funkcijom koja će vaš rdd_csv razdvojiti po stupcima te prikažite novi RDD na zaslonu.

In [22]:
rdd_csv = rdd_csv.map(lambda x: (x.split(",")[0], x.split(",")[1]))

rdd_csv.collect()

[('1677754800000', '23392.4024658030736446'),
 ('1677758400000', '23412.8301319667936481'),
 ('1677762000000', '23369.1657988242457019'),
 ('1677765600000', '23317.9928455364540223'),
 ('1677769200000', '23304.3620881745440072'),
 ('1677772800000', '23371.0505556610832414'),
 ('1677776400000', '23294.6043278283688753'),
 ('1677780000000', '23289.4984095429347556'),
 ('1677783600000', '23443.2121449701068961'),
 ('1677787200000', '23478.0257965128492930'),
 ('1677790800000', '23463.8413121360464792'),
 ('1677794400000', '23431.7409495071492525'),
 ('1677798000000', '23476.4063053381767990'),
 ('1677801600000', '23472.5406312534858921'),
 ('1677805200000', '23422.7917406744163382'),
 ('1677808800000', '22247.0716362180135542'),
 ('1677812400000', '22381.7864746646745348'),
 ('1677816000000', '22352.7524478982899446'),
 ('1677819600000', '22402.0371422146906710'),
 ('1677823200000', '22366.0191666461221423'),
 ('1677826800000', '22393.6259234369690590'),
 ('1677830400000', '22396.19939148

Transformacija reduce primjenjuje danu funkciju na dva parametra istog tipa te vraća jedan rezultat. Ta funkcija se primjenjuje elementima u strukturi sličnoj stablu, gdje se na svakoj razini broj elemenata prepolavlja sve dok se ne dođe do jednog elementa.

In [23]:
rdd_array.reduce(lambda x, y: x + y)

6

Transformacija groupByKey grupira zapise prema ključu i vraća RDD gdje svaki zapis ima ključ i listu vrijednosti povezanih s tim ključem.

In [24]:
rdd_employees = sc.parallelize([("Engineering", "Mike"), ("Engineering", "Tom"), ("Marketing", "Lisa")])

for key, values in rdd_employees.groupByKey().collect():
    print(f"{key}: {list(values)}")

Marketing: ['Lisa']
Engineering: ['Mike', 'Tom']


## Najviše dnevne cijene

Vaš zadatak je pronaći najveći dnevni maksimum kroz protekla dva tjedna koristeći RDD kao strukturu podataka. Pripazite na tipove učitanih podataka. Ovisno o kodu koji unesete možete nepovratno "pokvariti" kernel kada računate s RDD-om u Jupyteru pa ga stoga iznova pokrenite s vremena na vrijeme.

In [25]:
import datetime
from builtins import sum as builtin_sum

first_item = rdd_csv.first()

rdd_csv.map(lambda x: (datetime.datetime.fromtimestamp(int(x[0]) / 1000).date().strftime("%Y-%m-%d"), float(x[1]))) \
       .toDF(["date", "value"]) \
       .groupBy("date") \
       .agg(max(col("value"))) \
       .agg(max(col("max(value)"))) \
       .collect()


[Row(max(max(value))=28711.96546883728)]

# LAB4 task

In [26]:
df_bitcoin.write.format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5433/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "CryptoTable") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .mode("overwrite") \
    .save()

In [27]:
spark.read.format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5433/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "CryptoTable") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .load() \
    .show()

+-------------+------------------+----------+
|    timestamp|             value|      date|
+-------------+------------------+----------+
|1677754800000|23392.402465803072|2023-03-02|
|1677758400000|23412.830131966795|2023-03-02|
|1677762000000|23369.165798824244|2023-03-02|
|1677765600000|23317.992845536453|2023-03-02|
|1677769200000|23304.362088174545|2023-03-02|
|1677772800000|23371.050555661084|2023-03-02|
|1677776400000| 23294.60432782837|2023-03-02|
|1677780000000|23289.498409542935|2023-03-02|
|1677783600000|23443.212144970108|2023-03-02|
|1677787200000|23478.025796512848|2023-03-02|
|1677790800000|23463.841312136046|2023-03-02|
|1677794400000|23431.740949507148|2023-03-02|
|1677798000000|23476.406305338176|2023-03-03|
|1677801600000|23472.540631253487|2023-03-03|
|1677805200000|23422.791740674416|2023-03-03|
|1677808800000|22247.071636218014|2023-03-03|
|1677812400000|22381.786474664674|2023-03-03|
|1677816000000| 22352.75244789829|2023-03-03|
|1677819600000| 22402.03714221469|